# Étude d'une équation de convection scalaire

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
#
import pyfvm.mesh  as mesh
import pyfvm.model as model
import pyfvm.field as data
from pyfvm.xnum        import *
from pyfvm.integration import *

On cherche à résoudre l'évolution instationnaire du problème linéaire de convection suivant

$$ \frac{\partial q}{\partial t} + a \frac{\partial q}{\partial x} = 0 $$

pour la quantité transportée $q(x,t)$ et la condition intiale $q_0(x)$ sur le domaine $[0;\ell]$ avec des conditions périodiques. On choisit $\ell=1\rm~m$ et $a=1\rm~m/s$. 


## Définition des maillages, du modèle physique et solution initiales


In [6]:
mesh200 = mesh.unimesh(ncell=200, length=1.)
mesh100 = mesh.unimesh(ncell=100, length=1.)
mesh50  = mesh.unimesh(ncell=50,  length=1.)

mymodel = model.convmodel(1.)

# sinus packet
def init_sinpack(mesh):
    return np.sin(2*2*np.pi/mesh.length*mesh.centers())*(1+np.sign(-(mesh.centers()/mesh.length-.25)*(mesh.centers()/mesh.length-.75)))/2        
    
# periodic wave
def init_sinper(mesh):
    k = 2 # nombre d'onde
    return np.sin(2*k*np.pi/mesh.length*mesh.centers())
    
# square signal
def init_square(mesh):
    return (1+np.sign(-(mesh.centers()/mesh.length-.25)*(mesh.centers()/mesh.length-.75)))/2


## Calcul et comparaison

In [14]:
endtime = 5   # final physical time of simulation
ntime   = 1   # number of intermediate snapshots, only 1 is recommended
tsave   = np.linspace(0, endtime, num=ntime+1) 

initm   = init_square   # 
meshs   = [ mesh50 ]

cfls    = [ 0.5, 0.9, 1. ]

# schéma linéaire: extrapol1(), extrapol2()=extrapolk(1), centered=extrapolk(-1), extrapol3=extrapol(1./3.) 
# schéma avec limitation non linéaire: muscl(LIMITER) avec LIMITER = minmod, vanalbada, vanleer, superbee
xmeths  = [ extrapol1() ]  

# explicit, rk2, rk3ssp, rk4, implicit, trapezoidal=cranknicolson
tmeths  = [ rk4 ]

legends = [ 'CFL 0.5', '0.9', '1.', '1.1' ]

solvers = []
results = []   
nbcalc     = max(len(cfls), len(tmeths), len(xmeths), len(meshs))
for i in range(nbcalc):
    field0 = data.scafield(mymodel, 'p', (meshs*nbcalc)[i].ncell)
    field0.qdata[0] = initm((meshs*nbcalc)[i])
    solvers.append((tmeths*nbcalc)[i]((meshs*nbcalc)[i], (xmeths*nbcalc)[i]))
    start = time.clock()
    results.append(solvers[-1].solve(field0, (cfls*nbcalc)[i], tsave))
    print "cpu time of '"+legends[i]+" computation (",solvers[-1].nit,"it) :",time.clock()-start,"s"

# display results
style=['o', 'x', 'D', '*', 'o', 'o']
fig=plt.figure(1, figsize=(14,10))
plt.plot(meshs[0].centers(), results[0][0].qdata[0], '-')
labels = ["initial condition"]
for t in range(1,len(tsave)):
    for i in range(nbcalc):
        plt.plot((meshs*nbcalc)[i].centers(), results[i][t].qdata[0], style[i])
        labels.append(legends[i]+", t=%.1f"%results[i][t].time)
plt.legend(labels, loc='upper left',prop={'size':12})  


---

<a id="ipython"></a>
## Ipython et notebook : usage

* le notebook utilise la langage de base python en version améliorée, Ipython, qui permet la complétion des noms (variables, fonctions, modules) avec la touche tabulation
* toutes les cellules peuvent être modifiées par un double-clic et sont réinterprêtées avec `shift-entrée`
* l'ensemble de la feuille peut être exécutée avec le menu `Cell/run all cells`
* **n'oubliez pas de sauvegarder régulièrement votre feuille** (bouton _enregistrer_)


In [8]:
from IPython.core.display import HTML ; HTML(open("./custom.css", "r").read()) # notebook style